In [1]:
import sys
sys.path.append('..')
import doctable as dt
import spacy
from pprint import pprint
nlp = spacy.load('en')

## Store a Parsetree
To convert a sentence to its parsetree use the `DocParser.get_parsetrees()` and to store in a database use the parsetree column type.

In [3]:
# make a doctable db
schema = (
    ('id','integer',dict(primary_key=True, autoincrement=True)),
    ('tree','parsetree'),
)
db = dt.DocTable2(schema=schema)

# make a parsetree
sent = 'Barak Obama is the coolest cat out there. But wait - was he a good president?'
doc = nlp(sent)
parsetree = dt.DocParser.get_parsetrees(doc, merge_ents=True)

# insert and retreive the parsetree
db.insert({'tree': parsetree})
trees = db.select_first('tree')
trees

[ParseTree(8), ParseTree(9)]